In [1]:
import pandas as pd
import numpy as np

In [2]:
import allel # ref - https://scikit-allel.readthedocs.io/en/stable/io.html#variant-call-format-vcf

In [3]:
vcf_folder = "../dataset/"
vcf_file_freebayes = "syn2-freebayes.vcf.gz"
vcf_file_mutect = "syn2-mutect2.vcf.gz"
vcf_file_vardict = "syn2-vardict.vcf.gz"
vcf_file_varscan = "syn2-varscan.vcf.gz"

In [4]:
df_freebayes = allel.vcf_to_dataframe(vcf_folder + vcf_file_freebayes)

In [5]:
df_freebayes.head()

,CHROM,POS,ID,REF,ALT_1,ALT_2,ALT_3,QUAL,FILTER_PASS
0,1,10177,rs367896724,A,AC,NaN,NaN,317.100006,False
1,1,10352,rs555500075;rs145072688,T,TA,NaN,NaN,436.500000,False
2,1,10583,rs58108140,G,A,NaN,NaN,99.000000,False
3,1,10622,.,T,G,NaN,NaN,26.200001,False
4,1,10649,.,G,A,NaN,NaN,19.100000,False


In [6]:
df_mutect = allel.vcf_to_dataframe(vcf_folder + vcf_file_mutect)
df_mutect.head()

,CHROM,POS,ID,REF,ALT_1,ALT_2,ALT_3,QUAL,FILTER_PASS
0,1,137172,.,T,G,NaN,NaN,NaN,True
1,1,237752,rs374471060,A,G,NaN,NaN,NaN,False
2,1,237762,.,G,A,NaN,NaN,NaN,False
3,1,355088,.,TCTCCCCTG,T,NaN,NaN,NaN,False
4,1,355100,.,A,C,NaN,NaN,NaN,False


In [7]:
df_vardict = allel.vcf_to_dataframe(vcf_folder + vcf_file_vardict)
df_vardict.head()

,CHROM,POS,ID,REF,ALT_1,ALT_2,ALT_3,QUAL,FILTER_PASS
0,1,10177,rs367896724,A,AC,NaN,NaN,159.0,False
1,1,10231,rs200279319,C,A,NaN,NaN,114.0,False
2,1,10329,rs150969722,AC,A,NaN,NaN,146.0,False
3,1,10519,rs62636508,G,C,NaN,NaN,60.0,False
4,1,10583,rs58108140,G,A,NaN,NaN,92.0,False


In [8]:
df_varscan = allel.vcf_to_dataframe(vcf_folder + vcf_file_varscan)
df_varscan.head()

,CHROM,POS,ID,REF,ALT_1,ALT_2,ALT_3,QUAL,FILTER_PASS
0,1,10177,rs367896724,A,AC,NaN,NaN,0.0,False
1,1,10230,rs775928745,AC,A,NaN,NaN,0.0,False
2,1,10231,rs200279319,C,A,NaN,NaN,0.0,False
3,1,10234,rs145599635,C,T,NaN,NaN,0.0,False
4,1,10247,.,TA,T,NaN,NaN,0.0,False


In [9]:
print(f"Real freebayes shape = {df_freebayes.shape}")
print(f"Real Mutect shape = {df_mutect.shape}")
print(f"Real Vardict shape = {df_vardict.shape}")
print(f"Real Varscan shape = {df_varscan.shape}")

Real freebayes shape = (4986115, 9)
Real Mutect shape = (114403, 9)
Real Vardict shape = (5002549, 9)
Real Varscan shape = (4753923, 9)


In [10]:
m1 = pd.merge(df_freebayes, df_mutect, on = ["CHROM", "POS"], how="outer", suffixes = ("_freebayes", "_mutect"))

In [11]:
m1.shape

(5088347, 16)

In [12]:
m2 = pd.merge(m1, df_vardict, on = ["CHROM", "POS"], how="outer")
m2.rename(columns={"ID": "ID_vardict", "REF": "REF_vardict", "ALT_1": "ALT_1_vardict", "ALT_2": "ALT_2_vardict", \
                  "ALT_3": "ALT_3_vardict", "QUAL": "QUAL_vardict", "FILTER_PASS": "FILTER_PASS_vardict"}, inplace=True)

In [13]:
m2.head()

,CHROM,POS,ID_freebayes,REF_freebayes,ALT_1_freebayes,ALT_2_freebayes,ALT_3_freebayes,QUAL_freebayes,FILTER_PASS_freebayes,ID_mutect,...,ALT_3_mutect,QUAL_mutect,FILTER_PASS_mutect,ID_vardict,REF_vardict,ALT_1_vardict,ALT_2_vardict,ALT_3_vardict,QUAL_vardict,FILTER_PASS_vardict
0,1,10177,rs367896724,A,AC,NaN,NaN,317.100006,False,NaN,...,NaN,NaN,NaN,rs367896724,A,AC,NaN,NaN,159.0,False
1,1,10352,rs555500075;rs145072688,T,TA,NaN,NaN,436.500000,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,10583,rs58108140,G,A,NaN,NaN,99.000000,False,NaN,...,NaN,NaN,NaN,rs58108140,G,A,NaN,NaN,92.0,False
3,1,10622,.,T,G,NaN,NaN,26.200001,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,10649,.,G,A,NaN,NaN,19.100000,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
m2.shape

(5480261, 23)

In [15]:
m3 = pd.merge(m2, df_varscan, on = ["CHROM", "POS"], how="outer")
m3.rename(columns={"ID": "ID_varscan", "REF": "REF_varscan", "ALT_1": "ALT_1_varscan", "ALT_2": "ALT_2_varscan", \
                  "ALT_3": "ALT_3_varscan", "QUAL": "QUAL_varscan", "FILTER_PASS": "FILTER_PASS_varscan"}, inplace=True)
m3.shape

(5783210, 30)

In [16]:
m3.head()

,CHROM,POS,ID_freebayes,REF_freebayes,ALT_1_freebayes,ALT_2_freebayes,ALT_3_freebayes,QUAL_freebayes,FILTER_PASS_freebayes,ID_mutect,...,ALT_3_vardict,QUAL_vardict,FILTER_PASS_vardict,ID_varscan,REF_varscan,ALT_1_varscan,ALT_2_varscan,ALT_3_varscan,QUAL_varscan,FILTER_PASS_varscan
0,1,10177,rs367896724,A,AC,NaN,NaN,317.100006,False,NaN,...,NaN,159.0,False,rs367896724,A,AC,NaN,NaN,0.0,False
1,1,10352,rs555500075;rs145072688,T,TA,NaN,NaN,436.500000,False,NaN,...,NaN,NaN,NaN,rs555500075;rs145072688,T,TA,NaN,NaN,0.0,False
2,1,10583,rs58108140,G,A,NaN,NaN,99.000000,False,NaN,...,NaN,92.0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,10622,.,T,G,NaN,NaN,26.200001,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,10649,.,G,A,NaN,NaN,19.100000,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
m3.head()[["CHROM", "POS", "FILTER_PASS_freebayes", "FILTER_PASS_mutect", "FILTER_PASS_vardict", "FILTER_PASS_varscan"]]

,CHROM,POS,FILTER_PASS_freebayes,FILTER_PASS_mutect,FILTER_PASS_vardict,FILTER_PASS_varscan
0,1,10177,False,NaN,False,False
1,1,10352,False,NaN,NaN,False
2,1,10583,False,NaN,False,NaN
3,1,10622,False,NaN,NaN,NaN
4,1,10649,False,NaN,NaN,NaN


In [18]:
df_merged = m3

In [19]:
# add third category instead of False?
df_merged["FILTER_PASS_freebayes"].fillna(False, inplace=True)
df_merged["FILTER_PASS_vardict"].fillna(False, inplace=True)
df_merged["FILTER_PASS_mutect"].fillna(False, inplace=True)
df_merged["FILTER_PASS_varscan"].fillna(False, inplace=True)
df_merged.head()[["CHROM", "POS", "FILTER_PASS_freebayes", "FILTER_PASS_mutect", "FILTER_PASS_vardict", "FILTER_PASS_varscan"]]

,CHROM,POS,FILTER_PASS_freebayes,FILTER_PASS_mutect,FILTER_PASS_vardict,FILTER_PASS_varscan
0,1,10177,False,False,False,False
1,1,10352,False,False,False,False
2,1,10583,False,False,False,False
3,1,10622,False,False,False,False
4,1,10649,False,False,False,False


In [20]:
truth = pd.read_csv(vcf_folder + "syn2_truth.bed", sep = "\t", header = None)
truth.columns = ["CHROM", "POS_START", "POS_END"]
truth.head()

,CHROM,POS_START,POS_END
0,1,1787808,1787808
1,1,1819379,1819379
2,1,2115269,2115269
3,1,2422541,2422541
4,1,2494323,2494323


In [21]:
(truth.POS_START == truth.POS_END).sum()

4332

In [22]:
truth.shape

(4332, 3)

In [23]:
truth.merge(df_merged, left_on = ["CHROM", "POS_START"], right_on = ["CHROM", "POS"])
# this is smaller than the actual truth file indicating the all 4 variant callers missed to classify some true variants

,CHROM,POS_START,POS_END,POS,ID_freebayes,REF_freebayes,ALT_1_freebayes,ALT_2_freebayes,ALT_3_freebayes,QUAL_freebayes,...,ALT_3_vardict,QUAL_vardict,FILTER_PASS_vardict,ID_varscan,REF_varscan,ALT_1_varscan,ALT_2_varscan,ALT_3_varscan,QUAL_varscan,FILTER_PASS_varscan
0,1,1787808,1787808,1787808,.,C,T,NaN,NaN,339.399994,...,NaN,143.0,True,.,C,T,NaN,NaN,0.0,True
1,1,1819379,1819379,1819379,rs968266291,A,C,NaN,NaN,1611.099976,...,NaN,NaN,False,rs968266291,A,C,NaN,NaN,0.0,True
2,1,2115269,2115269,2115269,.,A,T,NaN,NaN,210.100006,...,NaN,118.0,True,.,A,T,NaN,NaN,0.0,True
3,1,2422541,2422541,2422541,.,G,C,NaN,NaN,404.100006,...,NaN,148.0,True,.,G,C,NaN,NaN,0.0,True
4,1,2494323,2494323,2494323,rs556710728,C,T,NaN,NaN,220.600006,...,NaN,115.0,True,rs556710728,C,T,NaN,NaN,0.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4309,X,153631641,153631641,153631641,.,A,T,NaN,NaN,338.399994,...,NaN,141.0,True,.,A,T,NaN,NaN,0.0,True
4310,X,154104082,154104082,154104082,.,A,T,NaN,NaN,68.300003,...,NaN,87.0,True,.,A,T,NaN,NaN,0.0,True
4311,X,154417237,154417237,154417237,.,A,C,NaN,NaN,242.100006,...,NaN,124.0,True,.,A,C,NaN,NaN,0.0,False
4312,X,154795622,154795622,154795622,.,G,C,NaN,NaN,307.200012,...,NaN,139.0,True,.,G,C,NaN,NaN,0.0,True


In [24]:
df_merged_with_truth = pd.merge(truth, df_merged, left_on=["CHROM", "POS_START"], right_on = ["CHROM", "POS"], how="outer")
df_merged_with_truth.head()

,CHROM,POS_START,POS_END,POS,ID_freebayes,REF_freebayes,ALT_1_freebayes,ALT_2_freebayes,ALT_3_freebayes,QUAL_freebayes,...,ALT_3_vardict,QUAL_vardict,FILTER_PASS_vardict,ID_varscan,REF_varscan,ALT_1_varscan,ALT_2_varscan,ALT_3_varscan,QUAL_varscan,FILTER_PASS_varscan
0,1,1787808.0,1787808.0,1787808.0,.,C,T,NaN,NaN,339.399994,...,NaN,143.0,True,.,C,T,NaN,NaN,0.0,True
1,1,1819379.0,1819379.0,1819379.0,rs968266291,A,C,NaN,NaN,1611.099976,...,NaN,NaN,False,rs968266291,A,C,NaN,NaN,0.0,True
2,1,2115269.0,2115269.0,2115269.0,.,A,T,NaN,NaN,210.100006,...,NaN,118.0,True,.,A,T,NaN,NaN,0.0,True
3,1,2422541.0,2422541.0,2422541.0,.,G,C,NaN,NaN,404.100006,...,NaN,148.0,True,.,G,C,NaN,NaN,0.0,True
4,1,2494323.0,2494323.0,2494323.0,rs556710728,C,T,NaN,NaN,220.600006,...,NaN,115.0,True,rs556710728,C,T,NaN,NaN,0.0,True


In [25]:
df_merged_with_truth.shape

(5783228, 32)

In [26]:
def set_true_label(row):
    if str(row) == "nan":
        return False
    else:
        return True

In [27]:
df_merged_with_truth["truth"] = df_merged_with_truth["POS_START"].apply(lambda row:set_true_label(row))
df_merged_with_truth.sum()

CHROM                    1111111111111111111111111111111111111111111111...
POS_START                                                   347100151025.0
POS_END                                                     347100151025.0
POS                                                      432657808287365.0
QUAL_freebayes                                                5342314496.0
FILTER_PASS_freebayes                                                14442
ALT_2_mutect                                                           0.0
ALT_3_mutect                                                           0.0
QUAL_mutect                                                            0.0
FILTER_PASS_mutect                                                   14265
ALT_2_vardict                                                          0.0
ALT_3_vardict                                                          0.0
QUAL_vardict                                                   670674304.0
FILTER_PASS_vardict      

In [28]:
def decide_prediction(row):
    if row["FILTER_PASS_freebayes"] + row["FILTER_PASS_mutect"] + row["FILTER_PASS_vardict"] + row["FILTER_PASS_varscan"] >= 2:
        return True
    else:
        return False

In [29]:
df_merged_with_truth["preds"] = df_merged_with_truth.apply(lambda row: decide_prediction(row), axis = 1)

In [30]:
df_merged_with_truth["preds"].sum()

10327

In [31]:
extras_in_truth = df_merged_with_truth[df_merged_with_truth.POS.isna()].index

In [32]:
df_merged_with_truth.drop(extras_in_truth, inplace=True)

In [33]:
def predict_label_three_best(row):
    if row["FILTER_PASS_mutect"] == False:
        return False
    if int(row["FILTER_PASS_freebayes"]) + int(row["FILTER_PASS_vardict"]) + int(row["FILTER_PASS_varscan"]) >= 2:
        return True
    return False
df_merged_with_truth["three_best_mutect_true"] = df_merged_with_truth[["FILTER_PASS_freebayes", "FILTER_PASS_mutect", "FILTER_PASS_vardict", "FILTER_PASS_varscan"]].apply(predict_label_three_best, axis = 1)

In [34]:
from sklearn.metrics import f1_score, precision_score, recall_score
print(f'F1 score = {f1_score(df_merged_with_truth["truth"], df_merged_with_truth["preds"])}')
print(f'Precision = {precision_score(df_merged_with_truth["truth"], df_merged_with_truth["preds"])}')
print(f'Recall = {recall_score(df_merged_with_truth["truth"], df_merged_with_truth["preds"])}')

F1 score = 0.5845229151014275
Precision = 0.4143507310932507
Recall = 0.9918868799258229


In [35]:
from sklearn.metrics import classification_report
print(classification_report(df_merged_with_truth["truth"], df_merged_with_truth["preds"]))

              precision    recall  f1-score   support

       False       1.00      1.00      1.00   5778896
        True       0.41      0.99      0.58      4314

    accuracy                           1.00   5783210
   macro avg       0.71      1.00      0.79   5783210
weighted avg       1.00      1.00      1.00   5783210



In [36]:
print(classification_report(df_merged_with_truth["truth"], df_merged_with_truth["three_best_mutect_true"]))

              precision    recall  f1-score   support

       False       1.00      1.00      1.00   5778896
        True       0.88      0.97      0.93      4314

    accuracy                           1.00   5783210
   macro avg       0.94      0.99      0.96   5783210
weighted avg       1.00      1.00      1.00   5783210

